In [1]:
import icecube
from icecube import icetray, dataio
from icecube.icetray import I3Tray
import glob
import os
from datetime import datetime
import json
import csv
import matplotlib.pyplot as plt
import numpy as np

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

In [2]:
class CheckRandomness(icetray.I3Module):
    def __init__(self, ctx):
        icetray.I3Module.__init__(self,ctx)
        self.AddParameter("minbias_list", "minbias_list", None)
        self.AddParameter("spefit_list", "spefit_list", None)

    def Configure(self):
        self.minbias_list = self.GetParameter("minbias_list")
        self.spefit_list = self.GetParameter("spefit_list")
        
    def Physics(self, frame):
        if frame["I3EventHeader"].sub_event_stream == "NullSplit":
            filters = frame["FilterMask_NullSplit0"]
            minbias = filters["FilterMinBias_13"].condition_passed
            self.minbias_list.append( (frame["I3EventHeader"].event_id, minbias) )
            if "SPEFitSingle" in frame:
                self.spefit_list.append( (frame["I3EventHeader"].event_id, frame["SPEFitSingle"].dir.zenith, frame["SPEFitSingle"].dir.azimuth) )
        #if frame["I3EventHeader"].sub_event_stream == "InIceSplit":
         #   filters = frame["FilterMask"]
          #  minbias = filters["FilterMinBias_13"].condition_passed
           # self.minbias_list.append( (frame["I3EventHeader"].event_id, minbias) )
            #if "SPEFitSingle" in frame:
             #   self.spefit_list.append( (frame["I3EventHeader"].event_id, frame["SPEFitSingle"].dir.zenith, frame["SPEFitSingle"].dir.azimuth) )
        


In [3]:
path_trigger = "20904_examples/IC86.2020_corsika.020904.000001.i3.zst"
paths = ["first_trial/", "second_trial/", "third_trial/"]
minbias = {1: [], 2: [], 3: []}
spefit_single = {1: [], 2: [], 3: []}
for i, path in enumerate(paths):
    tray = I3Tray()
    tray.Add("I3Reader", filename=path+"L2_sanity_trial"+str(i+1)+".i3.gz")
    tray.Add(CheckRandomness, minbias_list = minbias[i+1], spefit_list = spefit_single[i+1])
    tray.Execute()



NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


In [4]:
#print(minbias)
#print(spefit_single)

for key, item in minbias.items():
    print(key, len([x for x in item if x[1] == True]))

for key, item in spefit_single.items():
    print(key, len(item))

1 1760
2 1762
3 1768
1 0
2 0
3 0
